In [18]:
# Import libraries to use
import pandas as pd
import os
import datetime

In [19]:
# format right at the beginning how to view the float type
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

In [20]:
# --- Determining the path for each folder/file ---
raw_data_folder = '../../Tesla_Deliveries_Production/raw_data_'
processed_data_folder = '../../Tesla_Deliveries_Production/processed_data'

data_stock = os.path.join(raw_data_folder,'tesla_stock.parquet')
data_tesla = os.path.join(raw_data_folder,'tesla_deliveries_dataset_2015_2025.csv')

Tesla_kaggle Dataframe

In [21]:
# Loading dataframe and take the opportunity to view crucial information
try:
    df_tesla = pd.read_csv(data_tesla)

    print('--- Informations about dataframe ---\n')
    df_tesla.info()

    print('\n--- Firsts lines ---\n')
    print(df_tesla.head())

except Exception as e:
    print(f'occurred a error on try load the file: {e}')

--- Informations about dataframe ---

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2640 entries, 0 to 2639
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Year                  2640 non-null   int64  
 1   Month                 2640 non-null   int64  
 2   Region                2640 non-null   object 
 3   Model                 2640 non-null   object 
 4   Estimated_Deliveries  2640 non-null   int64  
 5   Production_Units      2640 non-null   int64  
 6   Avg_Price_USD         2640 non-null   float64
 7   Battery_Capacity_kWh  2640 non-null   int64  
 8   Range_km              2640 non-null   int64  
 9   CO2_Saved_tons        2640 non-null   float64
 10  Source_Type           2640 non-null   object 
 11  Charging_Stations     2640 non-null   int64  
dtypes: float64(2), int64(7), object(3)
memory usage: 247.6+ KB

--- Firsts lines ---

   Year  Month         Region    Model  Estimated_Deli

In [22]:
# adjusting the visualization of columns to make each one without whitespaces
df_tesla.columns = [column.strip() for column in df_tesla.columns]

# adjusting the format of columns to make each as 'Category'
columns_category = ['Region', 'Model', 'Source_Type']

for col in columns_category:
    df_tesla[col] = df_tesla[col].astype('category')

In [23]:
#Creating a new columns called 'Date', joining ['Year', 'Month'], and indexing it as index for the dataframe

df_tesla['Day'] = 1
df_tesla['Date'] = pd.to_datetime(df_tesla[['Year', 'Month', 'Day']])
df_tesla = df_tesla.drop(columns=['Day', 'Year', 'Month'])

df_tesla = df_tesla.set_index('Date')

print(df_tesla.head())

                   Region    Model  Estimated_Deliveries  Production_Units  \
Date                                                                         
2023-05-01         Europe  Model S                 17646             17922   
2015-02-01           Asia  Model X                  3797              4164   
2019-01-01  North America  Model X                  8411              9189   
2021-02-01  North America  Model 3                  6555              7311   
2016-12-01    Middle East  Model Y                 12374             13537   

            Avg_Price_USD  Battery_Capacity_kWh  Range_km  CO2_Saved_tons  \
Date                                                                        
2023-05-01      92,874.27                   120       704        1,863.42   
2015-02-01      62,205.65                    75       438          249.46   
2019-01-01     117,887.32                    82       480          605.59   
2021-02-01      89,294.91                   120       712          7

In [24]:
# Saving the dataframe, after my transformation and adjustment process

processed_file_path = os.path.join(processed_data_folder,'tesla_processed_dataset_2015_2025.parquet')

df_tesla.to_parquet(processed_data_folder)

print(f'Data transformed and saved in: {processed_file_path}')

Data transformed and saved in: ../../Tesla_Deliveries_Production/processed_data\tesla_processed_dataset_2015_2025.parquet


Stock Dataframe

In [26]:
# Loading dataframe and take the opportunity to view crucial information
try:
    df_stock = pd.read_parquet(data_stock)

    print('--- Informations about dataframe ---\n')
    df_stock.info()

    print('\n--- Firsts lines ---\n')
    print(df_stock.head())

except Exception as e:
    print(f'occurred a error on try load the file: {e}')

--- Informations about dataframe ---

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2735 entries, 2015-01-02 to 2025-11-14
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (Close, TSLA)   2735 non-null   float64
 1   (High, TSLA)    2735 non-null   float64
 2   (Low, TSLA)     2735 non-null   float64
 3   (Open, TSLA)    2735 non-null   float64
 4   (Volume, TSLA)  2735 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 128.2 KB

--- Firsts lines ---

Price      Close  High   Low  Open    Volume
Ticker      TSLA  TSLA  TSLA  TSLA      TSLA
Date                                        
2015-01-02 14.62 14.88 14.22 14.86  71466000
2015-01-05 14.01 14.43 13.81 14.30  80527500
2015-01-06 14.09 14.28 13.61 14.00  93928500
2015-01-07 14.06 14.32 13.99 14.22  44526000
2015-01-08 14.04 14.25 14.00 14.19  51637500
